# Recognition of American Sign Language via a Leap Motion

*by Matt Hehn, Ben Campbell, & Chris Cochran, December, 2017*

## Introduction

In October 2017, we participated in Colorado State University's Virtual Reality Hackathon. During the 48-hour hackathon, our 4-person team developed a prototype Virtual Reality (VR) experience meant to allow people to learn or practice American Sign Language. Traditionally, learning ASL requires someone already skilled to act as a teacher/tutor to the new learner, which limits the ability of many to learn the language. Additionally, children having to learn ASL as their first language have yet greater difficulties learning the language without another way to convey meaning.

Our project in the hackathon was designed to interpret sign language using a Leap Motion sensor mounted on the front of a VR headset, and provide active feedback to the user as to whether they had accurately mimicked a sign (and what it was, if they succeeed. A visual reference was shown in the VR environment). To aid in first-language learning, the system was designed to be able to enter a 'quiz' demo mode, which would ask the user to sign out a full word, at which point the simulation would spawn in a 3D model of the object the word refers to. We ultimately [won the hackathon](https://source.colostate.edu/sign-language-project-wins-second-annual-ram-hackathon/).

The key limitation of this demo was its accuracy. In the original Unity project, the C# code used to take data from the sensor was making a very naive comparison with a threshhold of allowed difference between a single "good" reference for a sign and the current position of the hand and fingers. Using this method, we were only able to get the prototype to accurately and regularly recognize roughly six letters. 

It was our aim in this project (and even during the hackathon, though time limitations made it impossible) to train a neural network to recognize and categorize correct signs to then be re-incorporated back into the VR project at some later date.

## Methods

Steps I took.  Resources I used, such as our textbook [Russell and Norvig, 2014]. 

Say in detail what each team member did.

## Results

Show all results.  Intermediate resultw might be shown in above Methods section.  Plots, tables, whatever.

## Conclusions

What I learned.

### References

* [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [chuck] me, fldfkafjlak

Your report should contain approximately 1,500 to 5,000 words, in markdown cells.  You can count words by running the following python code in your report directory.

In [2]:
import io
from nbformat import current
import glob
nbfile = glob.glob('*.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Example of Project Report.ipynb is 123
